In [10]:
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz, process

In [11]:
unmatched_companies = pd.read_excel("C:\\Users\\msnin\\OneDrive\\Desktop\\SPAC Research\\DataMerge.xlsx", sheet_name="Unmatched_Companies")
spac_alpha_data = pd.read_excel("C:\\Users\\msnin\\OneDrive\\Desktop\\SPAC Research\\SPACAlphadata.xlsx")
data_merge = pd.read_excel("C:\\Users\\msnin\\OneDrive\\Desktop\\SPAC Research\\DataMerge.xlsx", sheet_name="Companies_Saad")


In [12]:
def fuzzy_match_ticker(row, ticker_choices):
    # Extracting the text to fuzzy match. Strip removes any whitespace.
    ticker_match = process.extractOne(row['Ticker - SS'].strip(), ticker_choices, scorer=fuzz.ratio)
    
    # Checking the similarity scores (if it's 100, it's a perfect match)
    if ticker_match[1] == 100:
        return ticker_match[0]
    else:
        return None


In [13]:
spac_ticker_choices = spac_alpha_data['SPAC Ticker'].dropna().tolist()
unmatched_companies['MatchwSTicker'] = unmatched_companies.apply(fuzzy_match_ticker, ticker_choices=spac_ticker_choices, axis=1)


newco_ticker_choices = spac_alpha_data['Newco Ticker'].dropna().tolist()
unmatched_companies['MatchwNTicker'] = unmatched_companies.apply(fuzzy_match_ticker, ticker_choices=newco_ticker_choices, axis=1)


newco_warrant_ticker_choices = spac_alpha_data['Newco Warrant Ticker'].dropna().tolist()
unmatched_companies['MatchwNWarrantT'] = unmatched_companies.apply(fuzzy_match_ticker, ticker_choices=newco_warrant_ticker_choices, axis=1)


In [ ]:
# Cell 6: Update 'CIK Code' using vectorized operations

spac_alpha_data['CIK Code'] = spac_alpha_data['CIK Code'].combine_first(
    unmatched_companies.merge(
        data_merge[['Ticker', 'cik']], 
        left_on='MatchwSTicker', 
        right_on='Ticker', 
        how='left'
    )['cik']
)
spac_alpha_data['CIK Code'] = spac_alpha_data['CIK Code'].combine_first(
    unmatched_companies.merge(
        data_merge[['Ticker', 'cik']], 
        left_on='MatchwNTicker', 
        right_on='Ticker', 
        how='left'
    )['cik']
)
spac_alpha_data['CIK Code'] = spac_alpha_data['CIK Code'].combine_first(
    unmatched_companies.merge(
        data_merge[['Ticker', 'cik']], 
        left_on='MatchwNWarrantT', 
        right_on='Ticker', 
        how='left'
    )['cik']
)


In [ ]:
# Cell 7: Save the updated SPACAlphadata

spac_alpha_data.to_excel("Updated_SPACAlphadata4534.xlsx", index=False)